In [1]:
from utils.utils import *
from utils.utils_nn import *
from utils.Rnn_model import Rnn_model
# import kwargs dicts
from utils.kwargs_file import kwargs_neural_data_init  # , kwargs_rnn

import numpy as np
np.random.seed(seed())

import tensorflow as tf

import os

from tensorflow.contrib import rnn

import random
random.seed(seed())

import argparse

In [2]:
"""
Enable passing some keyword arguments from command line.
This does not affect the Jupyter notebook.
"""
# try:
parser = argparse.ArgumentParser()

parser.add_argument('--use_suggestions', action='store', dest='use_suggestions',
                    help='Should the algorithm use label suggestions '+\
                        '(False would mean using only the given labeled data)', 
                    type=str, 
                    default=None)

_StoreAction(option_strings=['--use_suggestions'], dest='use_suggestions', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help='Should the algorithm use label suggestions (False would mean using only the given labeled data)', metavar=None)

In [3]:
# results = parser.parse_args()
args, unknown = parser.parse_known_args()

if args.use_suggestions is not None:
    kwargs_neural_data_init.use_suggestions = str(args.use_suggestions)

In [4]:
# import the correct kwargs file, based on suggestion args
suggestion_convert_dict = \
    {'0.7': '07', 
     '0.8': '08', 
     '0.9': '09', 
     'F': '10', 
     'False': '10'}

suggestion_str = \
    suggestion_convert_dict[kwargs_neural_data_init.use_suggestions]

import_kwargs_str = \
    'from utils.kwargs_file_{} import kwargs_rnn_GRU, kwargs_rnn_LSTM'.format(suggestion_str)

exec(import_kwargs_str)  # execute import statement

In [5]:
x_feed_train, y_feed_train, x_feed_val, y_feed_val,\
    char_int, char_int_inv, label_int, label_int_inv, \
    statistics_dict =\
    data_load_preprocess(**kwargs_neural_data_init)

Using label suggestion data with similarity threshold of 0.8
The are 3565 observations
Sampling from allowed 96 labels
96 labels in the validation set, with
2111 potential observation to draw from.
480 observations sampled for validation
1631 observations for training
The ratio of validation to *training* is about 0.294


In [6]:
#### check if there are any strange splits between training and validation
#### e.g. labels (training and validation) don't match exactly
#### or not enough (or too many) observation were set aside for validation
y_train_counter = Counter(y_feed_train)
y_train_labs = list(y_train_counter.keys())
y_train_labs.sort()

y_val_counter = Counter(y_feed_val)
y_val_labs = list(y_val_counter.keys())
y_val_labs.sort()

assert y_train_labs==y_train_labs, 'Labels in training and validation set do not match!'

validation_training_ratios = []
for label in y_train_labs:
    val_num = y_val_counter[label]
    train_num = y_train_counter[label]
    ratio = float(val_num / train_num)
    validation_training_ratios.append(ratio)
#     print('{:.3f}'.format(ratio), label, val_num, train_num)

print('Minimal validation to trainig ration is {:.3f}'.format(min(validation_training_ratios)))
print('Maximal validation to trainig ration is {:.3f}'.format(max(validation_training_ratios)))

Minimal validation to trainig ration is 0.100
Maximal validation to trainig ration is 0.333


In [7]:
# returns np.arrays to feed into tf model
# training data
X_train, _, Y_train = index_transorm_xy(x=x_feed_train, 
                                        y=y_feed_train, 
                                        char_int=char_int, 
                                        label_int=label_int, 
                                        n_class=statistics_dict['n_class'])

# validation data
X_val, _, Y_val = index_transorm_xy(x=x_feed_val, 
                                    y=y_feed_val, 
                                    char_int=char_int, 
                                    label_int=label_int, 
                                    n_class=statistics_dict['n_class'])

# write a metadata file for embeddings visualizer and create path string
embed_vis_path = write_embeddings_metadata(log_dir=kwargs_rnn_GRU.log_dir, 
                                           dictionary=char_int, 
                                           file_name='metadata.tsv')

In [8]:
kwargs_feed_dict_train = {'x': X_train, 'y': Y_train}
kwargs_feed_dict_test = {'x': X_val, 'y': Y_val}

kwargs_rnn_GRU = nice_dict({**kwargs_rnn_GRU, **statistics_dict})
kwargs_rnn_LSTM = nice_dict({**kwargs_rnn_LSTM, **statistics_dict})

In [ ]:
"""
Train GRU and LSTM models
"""

In [ ]:
##### GRU model
hparam_str = make_hparam_string(**kwargs_rnn_GRU)

rnn_GRU_model = Rnn_model(hparam_str=hparam_str, 
                      embed_vis_path=embed_vis_path, 
                      feed_dict_train=kwargs_feed_dict_train, 
                      feed_dict_test=kwargs_feed_dict_test, 
                **kwargs_rnn_GRU
                      )

rnn_GRU_model.train()
rnn_GRU_model.close_session()

In [ ]:
##### LSTM model
hparam_str = make_hparam_string(**kwargs_rnn_LSTM)

rnn_LSTM_model = Rnn_model(hparam_str=hparam_str, 
                      embed_vis_path=embed_vis_path, 
                      feed_dict_train=kwargs_feed_dict_train, 
                      feed_dict_test=kwargs_feed_dict_test, 
                **kwargs_rnn_LSTM
                      )

rnn_LSTM_model.train()
rnn_LSTM_model.close_session()

In [ ]:
"""
Get MRR per ATC code CPU utilization
"""

In [9]:
##### GRU model
hparam_str = make_hparam_string(**kwargs_rnn_GRU)

rnn_GRU_model = Rnn_model(hparam_str=hparam_str, 
                      embed_vis_path=embed_vis_path, 
                      feed_dict_train=kwargs_feed_dict_train, 
                      feed_dict_test=kwargs_feed_dict_test, 
                **kwargs_rnn_GRU
                      )

rnn_GRU_model.restore(kwargs_rnn_GRU.log_dir+hparam_str)

Loading variables from rnn_final_batch/sim_08/GRU,bidir=T,noisy_tanh,learn_p=F,noise_alpha=0.9,noise_half_normal=F,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=128,l2_weight_reg=1.0E-03,target_rep_weight=0.5/
Loading successful


In [12]:
[accuracy, cost, recip_rank, top_k, logits] = \
    rnn_GRU_model.run_eval()

In [13]:
np.shape(logits)

(480, 96)

In [14]:
rnn_GRU_model.n_class

96

In [16]:
np.shape(kwargs_feed_dict_test['x'])

(480, 80)

In [17]:
np.shape(kwargs_feed_dict_test['y'])

(480, 96)

In [19]:
kwargs_feed_dict_test['y'][0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])

In [35]:
np.argmax(kwargs_feed_dict_test['y'][0])

65

In [30]:
max(
[np.argmax(kwargs_feed_dict_test['y'][i]) for i in range(np.shape(kwargs_feed_dict_test['y'])[0])]
)

95

In [31]:
max(
list(range(rnn_GRU_model.n_class))
)

95

In [34]:
np.argmax(logits[0])

65

In [37]:
label_int_inv[65]

'L04AX03'

In [38]:
label_int_inv[np.argmax(logits[0])]

'L04AX03'

In [40]:
np.shape(kwargs_feed_dict_test['y'])

numpy.ndarray

In [42]:
y_feed_val[0]

'L04AX03'

In [48]:
with tf.Session() as sess:
    print(
    rnn_GRU_model.get_reciprocal_rank(logits=logits, 
                                      targets=kwargs_feed_dict_test['y']).eval()
    )

[ 1.          1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          0.33333334
  1.          1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.          0.5
  1.          0.07142857  1.          1.          1.          1.          1.
  0.5         1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.          0.2
  1.          1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.          1.
 